<table style="border: none" align="left">
    <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/pmservice/cars-4-you/master/static/images/logo.png" width="200" alt="Icon"></th>
       <th style="border: none"><font face="verdana" size="5" color="black"><b>Business area prediction and action recommendation</b></th>
   </tr>
</table>

<img align=left src="https://github.com/pmservice/cars-4-you/raw/master/static/images/action.png" width="550" alt="Icon">

Contents
- [0. Setup](#setup)
- [1. Introduction](#introduction)
- [2. Load and explore data](#load)
- [3. Create an Apache Spark machine learning model](#model)
- [4. Store the model in the Watson Machine Learning repository](#persistence)
- [5. Deploy the model in the IBM Cloud](#persistence)
- [6. Score the model](#score)

**Note:** This notebook works correctly with kernel `Python 3.5 with Spark 2.1`.

<a id="setup"></a>
## 0. Setup

In this section please use below cell to upgrade the `watson-machine-learning-client`.

### Training data setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

- Create a Watson Machine Learning Service instance (a free plan is offered).
- Create a Compose for PostgreSQL instance.

### Package installation

In [78]:
!rm -rf $PIP_BUILD
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /usr/local/src/conda3_runtime.v48/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk->watson-machine-learning-client) (0.9.3)


In [79]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /usr/local/src/conda3_runtime.v48/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk->watson-machine-learning-client->ibm-ai-openscale) (0.9.3)


In [80]:
!pip install psycopg2-binary | tail -n 1

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


**Note**: Please restart the kernel (Kernel -> Restart)

<a id="introduction"></a>
## 1. Introduction

This notebook defines, trains and deploys the model that recommends specific Action for unstatisfied customers.

<a id="load"></a>
## 2. Load and explore data

In this section you will upload training data to the database, load it as an Apache Spark DataFrame and perform a basic exploration.

Read data into Spark DataFrame from CSV file and show sample record.

### 2.1 Load data from git

In [81]:
!wget https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/data/car_rental_training_data.csv

--2018-12-13 09:20:23--  https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/data/car_rental_training_data.csv
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/notebooks/data/car_rental_training_data.csv [following]
--2018-12-13 09:20:23--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/notebooks/data/car_rental_training_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79518 (78K) [text/plain]
Saving to: ‘car_rental_training_data.csv.2’

100%[======================================>] 79,518      --.-K/s   in 0.004s  

2018-

In [82]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.read.csv(path="car_rental_training_data.csv", sep=";", header=True, inferSchema=True)
df_data.head()

Row(ID=83, Gender='Female', Status='M', Children=2, Age=48.85, Customer_Status='Inactive', Car_Owner='Yes', Customer_Service='I thought the representative handled the initial situation badly.  The company was out of cars, with none coming in that day.  Then the representative tried to find us a car at another franchise.  There they were successful.', Satisfaction=0, Business_Area='Product: Availability/Variety/Size', Action='Free Upgrade')

### 2.2 Upload data to PostgreSQL database.

**TIP:** Put your service credentials here. Just copy paste content of Credentials tab from service details (IBM Cloud)

In [83]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "name": "<put correct name here>",
    "uri": "<put correct uri here>"
}

In [84]:
# The code was removed by Watson Studio for sharing.

Put your schema name here.

In [8]:
SCHEMA_NAME = 'data_mart_for_wml'

You can use the code below to create the schema. If the schema already exists skipp this cell.

In [26]:
from ibm_ai_openscale.utils import create_postgres_schema

create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

Create training table.

In [11]:
TABLE_NAME = "CAR_RENTAL_TRAINING"

In [15]:
import psycopg2

hostname = POSTGRES_CREDENTIALS['uri'].split('@')[1].split(':')[0]
port = POSTGRES_CREDENTIALS['uri'].split('@')[1].split(':')[1].split('/')[0]
user = POSTGRES_CREDENTIALS['uri'].split('@')[0].split('//')[1].split(':')[0]
password = POSTGRES_CREDENTIALS['uri'].split('@')[0].split('//')[1].split(':')[1]
dbname = 'compose'
conn_string = "host=" + hostname + " port=" + port + " dbname=" + dbname + " user=" + user + " password=" + password


conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE {}.{}(
    ID integer,
    Gender text,
    Status text,
    Children integer,
    Age decimal(14,6),
    Customer_Status text,
    Car_Owner text,
    Customer_Service text,
    Satisfaction integer,
    Business_Area text,
    Action text
)
""".format(SCHEMA_NAME, TABLE_NAME))
cursor.close()
conn.close()

Upload training data to created table.

In [17]:
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
with open('car_rental_training_data.csv', 'r') as f:
    next(f)
    cursor.copy_from(file=f, table='{}.CAR_RENTAL_TRAINING'.format(SCHEMA_NAME), sep=';')
cursor.close()
conn.close()

Print first row in table.

In [18]:
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("""
SELECT * FROM {}.CAR_RENTAL_TRAINING LIMIT 1
""".format(SCHEMA_NAME))
result = cursor.fetchall()
cursor.close()
conn.close()

print(result)

[(83, 'Female', 'M', 2, Decimal('48.850000'), 'Inactive', 'Yes', 'I thought the representative handled the initial situation badly.  The company was out of cars, with none coming in that day.  Then the representative tried to find us a car at another franchise.  There they were successful.', 0, 'Product: Availability/Variety/Size', 'Free Upgrade')]


### 2.3 Explore data

In [85]:
df_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Children: integer (nullable = true)
 |-- Age: double (nullable = true)
 |-- Customer_Status: string (nullable = true)
 |-- Car_Owner: string (nullable = true)
 |-- Customer_Service: string (nullable = true)
 |-- Satisfaction: integer (nullable = true)
 |-- Business_Area: string (nullable = true)
 |-- Action: string (nullable = true)



**Tip:** Code above can be inserted using Data menu.  You have to select `Insert SparkSession DataFrame` option.

**Note:** Inserted code is modified to work with code in cells below.

As you can see, the data contains eleven fields. `Action` field is the one you would like to predict using feedback data in `Customer_Service` field.

In [28]:
print("Number of records: " + str(df_data.count()))

Number of records: 486


In [29]:
df_data.select('Business_area').groupBy('Business_area').count().show(truncate=False)

+----------------------------------+-----+
|Business_area                     |count|
+----------------------------------+-----+
|Service: Accessibility            |26   |
|Product: Functioning              |150  |
|Service: Attitude                 |24   |
|Service: Orders/Contracts         |32   |
|Product: Availability/Variety/Size|42   |
|Product: Pricing and Billing      |24   |
|Product: Information              |8    |
|Service: Knowledge                |180  |
+----------------------------------+-----+



In [30]:
df_data.select('Action').groupBy('Action').count().show(truncate=False)

+-------------------------+-----+
|Action                   |count|
+-------------------------+-----+
|NA                       |274  |
|Voucher                  |42   |
|Premium features         |30   |
|On-demand pickup location|56   |
|Free Upgrade             |84   |
+-------------------------+-----+



<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section you will learn how to:

- [3.1 Prepare data for training a model](#prep)
- [3.2 Create an Apache Spark machine learning pipeline](#pipe)
- [3.3 Train a model](#train)

<a id="prep"></a>
### 3.1 Prepare data for training a model

In this subsection you will split your data into: train and test data set.

In [31]:
train_data, test_data = df_data.randomSplit([0.8, 0.2], 24)

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 391
Number of testing records : 95


### 3.2 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline and then train the model.

In [32]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler, HashingTF, IDF, Tokenizer, SQLTransformer
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model, PipelineModel

In the following step, use the StringIndexer transformer to convert all the string fields to numeric ones.

In [33]:
tokenizer = Tokenizer(inputCol="Customer_Service", outputCol="words")

In [34]:
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='hash')
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="area_features", minDocFreq=5)

In [35]:
string_indexer_area = StringIndexer(inputCol="Business_Area", outputCol="area_label").fit(df_data)

In [36]:
dt_area = DecisionTreeClassifier(labelCol='area_label', featuresCol=idf.getOutputCol() , predictionCol='prediction_area', probabilityCol='probability_area', rawPredictionCol='rawPrediction_area')

Finally, convert the indexed labels back to original labels.

In [37]:
labelConverter = IndexToString(inputCol="prediction_area", outputCol="predictedAreaLabel", labels=string_indexer_area.labels)

In [38]:
string_indexer_gender = StringIndexer(inputCol="Gender", outputCol="gender_ix")
string_indexer_customer_status = StringIndexer(inputCol="Customer_Status", outputCol="customer_status_ix")
string_indexer_status = StringIndexer(inputCol="Status", outputCol="status_ix")
string_indexer_owner = StringIndexer(inputCol="Car_Owner", outputCol="owner_ix")

In [39]:
assembler = VectorAssembler(inputCols=["gender_ix", "customer_status_ix", "status_ix", "owner_ix", "Children", "Age", "Satisfaction", idf.getOutputCol()], outputCol="features")

In [40]:
string_indexer_action = StringIndexer(inputCol="Action", outputCol="label").fit(df_data)

In [41]:
label_action_converter = IndexToString(inputCol="prediction", outputCol="predictedActionLabel", labels=string_indexer_action.labels)

In [42]:
dt_action = DecisionTreeClassifier()

In [43]:
vector_assembler = VectorAssembler(inputCols=["gender_ix", "customer_status_ix", "status_ix", "owner_ix", "Children", "Age", "Satisfaction", 'prediction_area'], outputCol="features")

In [44]:
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, string_indexer_area, dt_area, labelConverter, string_indexer_gender, string_indexer_customer_status, string_indexer_status, string_indexer_action, string_indexer_owner, vector_assembler, dt_action, label_action_converter])

### 3.3 Train the model

**Hint:** Training takes ~10 minutes to complete.

In [45]:
model = pipeline.fit(train_data)

### 3.4 Evaluate the model

In [46]:
predictions = model.transform(test_data)

In [47]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.789474


<a id="persistence"></a>
## 4. Store the model in the repository

In this section you will store trained model to Watson Machine Learning repository. When model is stored some metada is optional, however we provide it to be able to configure Continuous Learning System.

In [48]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

We need Watson Machine Learning credentials to be able to store model in repository.

**TIP:** Put watson Machine Learning service credentials here.

In [49]:
WML_CREDENTIALS = {
  "apikey": "***",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "***",
  "iam_serviceid_crn": "***",
  "instance_id": "***",
  "password": "***",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "***"
}

In [87]:
# The code was removed by Watson Studio for sharing.

In [88]:
client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [89]:
client.version

'1.0.347'

### 4.2 Save the pipeline and model<a id="save"></a>

In [90]:
TRAINING_DATA_REFERENCE = {
 "name": "CARS4U training reference",
 "connection": POSTGRES_CREDENTIALS,
 "source": {
  "tablename": TABLE_NAME,
  "type": "postgres"
 }
}

Define `output_data_schema` for the model

In [91]:
train_data_schema = train_data.schema
label_field = next(f for f in train_data_schema.fields if f.name == "Action")
label_field.metadata['values'] = string_indexer_action.labels

In [92]:
from pyspark.sql.types import *

input_fileds = filter(lambda f: f.name != "Action", train_data_schema.fields)

OUTPUT_DATA_SCHEMA = StructType(list(input_fileds)). \
    add("prediction", DoubleType(), True, {'modeling_role': 'prediction'}). \
    add("predictedActionLabel", StringType(), True, {'modeling_role': 'decoded-target', 'values': string_indexer_action.labels}). \
    add("probability", ArrayType(DoubleType()), True, {'modeling_role': 'probability'})

Define model's metadata.

In [93]:
model_props = {
    client.repository.ModelMetaNames.NAME: "CARS4U - Action Recommendation Model",
    client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: TRAINING_DATA_REFERENCE,
    client.repository.ModelMetaNames.EVALUATION_METHOD: "multiclass",
    client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: OUTPUT_DATA_SCHEMA.jsonValue(),
    client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.7
        }
    ]
}

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available meta names.

Store the model.

In [94]:
published_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)

In [95]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

67d1d82d-eeb0-488a-b403-1a8dc00db72f


<a id="deploy"></a>
## 5. Deploy model in the IBM Cloud

You can use following command to create online deployment in cloud.

In [96]:
deployment_details = client.deployments.create(model_uid=model_uid, name='CARS4U - Area and Action Model Deployment')



#######################################################################################

Synchronous deployment creation for uid: '67d1d82d-eeb0-488a-b403-1a8dc00db72f' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='60e6fc28-bd6a-43b7-8b45-5eaedb2a7a79'
------------------------------------------------------------------------------------------------




You can use deployed model to score new data using scoring endpoint.

In [97]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/4d7a90a8-9df1-45be-be13-cb54930512cf/deployments/60e6fc28-bd6a-43b7-8b45-5eaedb2a7a79/online


<a id="score"></a>
## 6. Score the model

In [98]:
fields = ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status','Car_Owner', 'Customer_Service', 'Business_Area', 'Satisfaction']
values = [3785, 'Male', 'S', 1, 17, 'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 'Product: Information', 0]

In [99]:
payload_scoring = {"fields": fields,"values": [values]}
scoring_response = client.deployments.score(scoring_url, payload_scoring)

In [100]:
area_index = scoring_response['fields'].index('predictedAreaLabel')
action_index = scoring_response['fields'].index('predictedActionLabel')

print("Predicted Business Area: " + json.dumps(scoring_response['values'][0][area_index]))
print("Recommended action: " + json.dumps(scoring_response['values'][0][action_index]))

Predicted Business Area: "Service: Knowledge"
Recommended action: "On-demand pickup location"


---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.